In [38]:
from numpy import arange,linspace,var,logspace,log10 as log
from DiSuQ.utils import plotCompare
import pandas,torch,numpy

# GPU-CPU bottleneck

In [78]:
dGPUCPU = pandas.read_csv('./sparse-dense-gpu-re.csv')

In [90]:
dGPUCPU = pandas.read_csv('./sparse-dense-gpu.csv')
dGPUCPU1 = pandas.read_csv('./sparse-dense-gpu-small.csv')

In [91]:
dGPUCPU = pandas.concat([dGPUCPU1,dGPUCPU])

In [92]:
dGPUCPU.head()

,Unnamed: 0,lib,stage,hilbert,threads,peak_memory,time,vectors,rep
0,0,torch,diag,100,16,290.964844,0.776427,False,dense
1,1,torch,diag,100,16,41.066406,0.286004,False,sparse
2,2,torch,diag,167,16,0.003906,0.291159,False,dense
3,3,torch,diag,167,16,0.003906,0.314172,False,sparse
4,4,torch,diag,281,16,0.003906,0.295462,False,dense


In [93]:
hilbert = dGPUCPU['hilbert'].unique()
threads = dGPUCPU['threads'].unique()

In [94]:
ComputationSparse = dict(); ComputationDense = dict()
MemorySparse = dict(); MemoryDense = dict()
for thread in threads:
    ComputationDense[str(thread)] = (dGPUCPU[(dGPUCPU['threads']==thread) & (dGPUCPU['rep']=='dense')]['time'])
    ComputationSparse[str(thread)] = (dGPUCPU[(dGPUCPU['threads']==thread) & (dGPUCPU['rep']=='sparse')]['time'])

In [95]:
plotCompare(hilbert,ComputationSparse,None,'basis size','time(sec)',size=(600,600))

In [96]:
plotCompare(hilbert,ComputationDense,None,'basis size','time(sec)',size=(600,600))

## Dense Diagonalization 

# Sparse

In [14]:
dSpaDen1.head()

,Unnamed: 0,lib,stage,hilbert,threads,peak_memory,time,vectors,rep
0,0,torch,diag,100,16,0.074219,0.174287,False,dense
1,1,torch,diag,100,16,0.007812,0.150272,False,sparse
2,2,torch,diag,188,16,0.312500,0.211637,False,dense
3,3,torch,diag,188,16,0.003906,0.284335,False,sparse
4,4,torch,diag,354,16,0.230469,0.434264,False,dense


In [84]:
dSpaDen = pandas.read_csv('sparse-dense-re.csv')

In [97]:
dSpaDen = pandas.read_csv('sparse-dense.csv')
dSpaDenSmall = pandas.read_csv('sparse-dense-small.csv')

In [98]:
dSpaDen = pandas.concat([dSpaDenSmall,dSpaDen])

In [99]:
hilbert = dSpaDen['hilbert'].unique()
threads = dSpaDen['threads'].unique()

In [100]:
hilbert

array([ 100,  167,  281,  473,  794, 1000, 1333, 1778, 2371, 3162])

In [101]:
ComputationSparse = dict(); ComputationDense = dict()
MemorySparse = dict(); MemoryDense = dict()
for thread in threads:
    ComputationDense[str(thread)] = dSpaDen[(dSpaDen['threads']==thread) & (dSpaDen['rep']=='dense')]['time']
    ComputationSparse[str(thread)] = dSpaDen[(dSpaDen['threads']==thread) & (dSpaDen['rep']=='sparse')]['time']

In [102]:
plotCompare(hilbert,ComputationSparse,None,'basis size','time(sec)',size=(600,600))

In [103]:
plotCompare(hilbert,ComputationDense,None,'basis size','time(sec)',size=(600,600))

# Parallelization

In [28]:
dPara = pandas.read_csv('parallelization.csv')

In [29]:
dNum = dLog[dLog['lib']=='numpy']
dTor = dLog[dLog['lib']=='torch']

In [30]:
dNum.groupby(['threads','hilbert'])['time'].apply(list)

threads  hilbert
16       10         [0.32048748806118965]
         56          [0.2620991840958595]
         316        [0.24262834712862968]
         1778        [0.5671932324767113]
         10000          [42.567871209234]
32       10          [0.2676594518125057]
         56          [0.2741946466267109]
         316        [0.29015256091952324]
         1778        [0.5784363113343716]
         10000        [34.15516912564635]
64       10          [0.2774595655500889]
         56         [0.27154920995235443]
         316        [0.28771113976836205]
         1778        [0.7344554513692856]
         10000       [43.694167755544186]
Name: time, dtype: object

In [31]:
dTor.groupby(['threads','hilbert'])['time'].apply(list)

threads  hilbert
16       10          [0.3507659137248993]
         56           [0.360497422516346]
         316        [0.24776870384812355]
         1778       [0.42042311280965805]
         10000        [77.66510659456253]
32       10         [0.40429428964853287]
         56         [0.23862718418240547]
         316        [0.23803506419062614]
         1778        [0.4247867427766323]
         10000       [53.645921275019646]
64       10         [0.43307001143693924]
         56          [0.2745783366262913]
         316        [0.28994231671094894]
         1778        [0.5305673442780972]
         10000        [85.90453988313675]
Name: time, dtype: object

In [ ]:
plotCompare(bases,{'numpy':dNum[dLog['']]})

# Multi-Node GPU

# I/O bottleneck

# GPU advantage